In [2]:
import tensorflow as tf
import numpy as np
tf.debugging.set_log_device_placement(True)


In [3]:
tf.executing_eagerly()

True

In [4]:
tf.test.is_gpu_available()

True

In [ ]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))


In [5]:
dim1 = 1000
dim2 = 2000
import time
# d = "/device:XLA_"
d = ""
worker0 = d + "GPU:0"
worker1 = d + "GPU:1"
worker2 = d + "GPU:2"
worker3 = d + "GPU:3"

In [7]:
from numba import cuda 
cuda.select_device(0)
cuda.close()

In [ ]:
import tensorflow as tf

from multiprocessing import Pool

def _process(dim1, dim2):
    a1 = tf.random.uniform([dim1,dim2])
    return tf.matmul(a1, tf.transpose(a1))
    
def process_image(image):
    with Pool(1) as p:
        return p.apply(_process, (dim1, dim2))

In [ ]:
tf.debugging.set_log_device_placement(True)
t0 = time.time()    

with tf.device(worker0):
  a1 = tf.random.uniform([dim1,dim2])
  b1 = tf.matmul(a1, tf.transpose(a1))

with tf.device(worker1):
  a2 = tf.random.uniform([dim1,dim2])
  b2 = tf.matmul(a2, tf.transpose(a2))

with tf.device(worker2):
  a3 = tf.random.uniform([dim1,dim2])
  b3 = tf.matmul(a3, tf.transpose(a3))

with tf.device(worker3):
  result = b1+b2+b3
#   print(result.numpy())

t1 = time.time()
print(f"total timing is {str(t1-t0)}")


In [7]:
tf.debugging.set_log_device_placement(True)
t0 = time.time()    
with tf.device(worker0):
    a1 = tf.random.uniform([dim1,dim2])
    b1 = tf.matmul(a1, tf.transpose(a1))
    a2 = tf.random.uniform([dim1,dim2])
    b2 = tf.matmul(a2, tf.transpose(a2))
    a3 = tf.random.uniform([dim1,dim2])
    b3 = tf.matmul(a3, tf.transpose(a3))

    result = b1+b2 +b3
    #   print(result.numpy())

t1 = time.time()
print(f"total timing is {str(t1-t0)}")


Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Transpose in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
total timing is 0.04947686195373535


In [10]:
import time

cpu_slot = 0
gpu_slot = 0

# Using CPU at slot 0
with tf.device('/CPU:' + str(cpu_slot)):
    # Starting a timer
    start = time.time()

    # Doing operations on CPU
    A = tf.constant([[3, 2], [5, 2]])
    print(tf.eye(2,2))

    # Printing how long it took with CPU
    end = time.time() - start
    print(end)

# Using the GPU at slot 0
with tf.device('/GPU:' + str(gpu_slot)):
    # Starting a timer
    start = time.time()

    # Doing operations on CPU
    A = tf.constant([[3, 2], [5, 2]])
    print(tf.eye(2,2))

    # Printing how long it took with CPU
    end = time.time() - start
    print(end)

Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatrixDiag in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[1. 0.]
 [0. 1.]], shape=(2, 2), dtype=float32)
0.002477884292602539
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatrixDiag in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[1. 0.]
 [0. 1.]], shape=(2, 2), dtype=float32)
0.001949310302734375
